In [95]:
import pickle as pkl
import numpy as np

In [96]:
input_dim = 320

In [97]:
with open('Train_'+str(input_dim)+'.pkl', 'rb') as tr:
    X_train = pkl.load(tr)
tr.close()
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(87719, 320)

In [98]:
with open('Val_'+str(input_dim)+'.pkl', 'rb') as val:
    X_val = pkl.load(val)
val.close()
# X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_val.shape

(998, 320)

In [99]:
with open('Test_'+str(input_dim)+'.pkl', 'rb') as test:
    X_test = pkl.load(test)
test.close()
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test.shape

(999, 320)

In [100]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87719

In [101]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

998

In [102]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

999

In [103]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [104]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87719,)

In [105]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(998,)

In [106]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(999,)

In [107]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping

In [108]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87719, 10)

In [109]:
y_val = to_categorical(y_val_labels)
y_val.shape

(998, 10)

In [110]:
y_test = to_categorical(y_test_labels)
y_test.shape

(999, 10)

In [111]:
from sklearn.utils.class_weight import compute_class_weight

In [112]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [113]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [290]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

In [291]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(320,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [292]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_92 (Dense)             (None, 16)                5136      
_________________________________________________________________
dense_93 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_94 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_95 (Dense)             (None, 10)                170       
Total params: 5,850
Trainable params: 5,850
Non-trainable params: 0
_________________________________________________________________
None


In [293]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot feedforward.png', show_shapes=True, show_layer_names=True)

In [294]:
earlystop = EarlyStopping(monitor='val_acc', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [295]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=(X_val, y_val), class_weight = class_weights_dict, verbose=1)

Train on 87719 samples, validate on 998 samples
Epoch 1/100
87719/87719 [==============================] - 3s 32us/step - loss: 1.9987 - acc: 0.4797 - val_loss: 1.5353 - val_acc: 0.4810
Epoch 2/100
87719/87719 [==============================] - 1s 11us/step - loss: 1.3588 - acc: 0.6499 - val_loss: 1.1764 - val_acc: 0.6032
Epoch 3/100
87719/87719 [==============================] - 1s 11us/step - loss: 1.1296 - acc: 0.6941 - val_loss: 1.0432 - val_acc: 0.6593
Epoch 4/100
87719/87719 [==============================] - 1s 10us/step - loss: 1.0019 - acc: 0.7244 - val_loss: 0.9529 - val_acc: 0.6974
Epoch 5/100
87719/87719 [==============================] - 1s 10us/step - loss: 0.9165 - acc: 0.7442 - val_loss: 0.8819 - val_acc: 0.7275
Epoch 6/100
87719/87719 [==============================] - 1s 10us/step - loss: 0.8590 - acc: 0.7568 - val_loss: 0.8418 - val_acc: 0.7345
Epoch 7/100
87719/87719 [==============================] - 1s 10us/step - loss: 0.8198 - acc: 0.7660 - val_loss: 0.8164 - va

In [296]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 0.6696155535911988
Val accuracy: 0.7955911823647295


In [297]:
y_pred = model.predict_classes(X_val)
confusion_matrix(y_pred, y_val_labels)

array([[68,  1,  3,  2,  0,  0,  5,  3,  2,  0],
       [ 4, 81,  3,  7,  2,  0,  1,  4,  2,  5],
       [ 6,  4, 84,  2,  1,  0,  1,  2,  1,  0],
       [ 9,  2,  0, 80,  0,  2,  2,  4,  0,  2],
       [ 1,  2,  1,  1, 86,  2,  0,  1,  7,  5],
       [ 0,  1,  0,  0,  1, 92,  1,  5,  1,  2],
       [ 2,  1,  0,  2,  0,  1, 78,  4,  2,  2],
       [ 5,  5,  5,  3,  0,  0,  5, 70,  1,  2],
       [ 1,  2,  1,  3,  5,  1,  4,  4, 75,  2],
       [ 4,  1,  3,  0,  5,  2,  2,  2,  9, 80]], dtype=int64)

In [298]:
test_scores = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.7781983764769437
Test accuracy: 0.7597597597597597


In [299]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_pred, y_test_labels)

array([[79,  2,  4,  3,  2,  1,  6,  6,  0,  2],
       [ 5, 78,  3,  1,  0,  2,  0,  5,  5,  6],
       [ 3,  3, 81,  1,  2,  0,  2,  4,  1,  1],
       [ 3,  6,  3, 76,  0,  0,  3,  5,  0,  0],
       [ 1,  2,  0,  1, 80,  1,  0,  1, 12,  4],
       [ 1,  0,  1,  2,  0, 87,  4,  2,  0,  3],
       [ 3,  1,  0,  6,  0,  2, 72,  5,  0,  1],
       [ 4,  1,  2,  7,  0,  4,  7, 63,  3,  3],
       [ 0,  5,  3,  0,  6,  0,  3,  0, 69,  6],
       [ 1,  2,  3,  3, 10,  3,  3,  8, 10, 74]], dtype=int64)